# Подготвка данных x_new и предсказание лучших моделей.
_______


In [2]:
!pip install catboost
!pip install xgboost

     |████████████████████████████████| 67.4 MB 29 kB/s 


In [3]:
import warnings
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.metrics import roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.metrics import plot_confusion_matrix, roc_curve, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_SEED = 42
warnings.filterwarnings("ignore")

In [8]:
X = pd.read_excel('/content/drive/MyDrive/TЗ_Будущее/x_train.xlsx')
y = pd.read_excel('/content/drive/MyDrive/TЗ_Будущее/y_train.xlsx')
x_new = pd.read_excel('/content/drive/MyDrive/TЗ_Будущее/x_new.xlsx')

berth_date = ['день рождения клиента',
              'месяц рождения клиента', 'возраст клиента']
contr_date = ['лет с заключения договора',
              'месяцев с заключения договора', 'дней с заключения договора']
dath_date = ['лет на момент смерти',
             'месяцев на момент смерти', 'дней на момент смерти']

In [9]:
X['Train'] = True  # помечаем где у нас трейн
x_new['Train'] = False  # помечаем где у нас тест
X = pd.concat([X, x_new], ignore_index=True)  # Объединяем

In [10]:
# Объеденим несколько признаков во едино.
X['contract_days'] = X.apply(lambda row: row[contr_date[0]]*365 +
                             row[contr_date[1]]*30 +
                             row[contr_date[2]], axis=1)

X['live_days'] = X.apply(lambda row: row[dath_date[0]]*365 +
                         row[dath_date[1]]*30 +
                         row[dath_date[2]], axis=1)

# удалим лишние столбцы ['лет с заключения договора', 'месяцев с заключения договора', 'дней с заключения договора']
X.drop(contr_date, axis=1, inplace=True)
X.drop(dath_date, axis=1, inplace=True)

X['contract_days'] = np.log(X['contract_days'] + 1)

X['live'] = (X['live_days'] == 0).map(int)
X['have_maney'] = (X['баланс счета'] == 0).map(int)

# Отцефруем бинарные признаки
bin_cols = ['инвестор', 'пол клиента']
label_encoder = LabelEncoder()
for col in bin_cols:
    X[col] = label_encoder.fit_transform(X[col])

In [11]:
X_train = X[X.Train].drop('Train', axis=1)
X_test = X[~X['Train']].drop('Train', axis=1)
y_train_type = y.iloc[:, 2]  # берем тип выплаты

# Предсказываем тип выплаты
model_2 = CatBoostClassifier(
    iterations=1100,
    learning_rate=0.5,
    loss_function="MultiClass",
    random_state=42,
    verbose=0,
    #                           thread_count=4,
    depth=3,
    l2_leaf_reg=3.28,
)

model_2.fit(X_train, y_train_type)
y_pred_type = model_2.predict(X_test)
y_pred_type = y_pred_type.flatten()

# Предсказываем месяц выплаты
y_train_month = y.iloc[:, 0]

"""добавление параметров objective="multi:softmax",  num_class=12 на качество
 не повлияло"""
xgb = XGBClassifier(max_depth=3, n_estimators=1000, learning_rate=0.1,
                    objective="multi:softmax",  num_class=12)
xgb.fit(X_train, y_train_month)
y_pred_month = xgb.predict(X_test)

# Предсказываем с сумму выплаты
X_1 = X.copy()

# Добавим новые признаки - выполним разбение значений некоторых столбцов по 10 категорий.
X_1['contract_days_bins'] = pd.cut(X_1['contract_days'], bins=10, labels=[
                                   str(x) for x in range(1, 11)]).astype(int)
X_1['live_days_bins'] = pd.cut(X_1['баланс счета'], bins=10, labels=[
                               str(x) for x in range(1, 11)]).astype(int)

X_train_1 = X_1[X.Train].drop('Train', axis=1)
X_test_1 = X_1[~X_1['Train']].drop('Train', axis=1)
y_train_sum = y.iloc[:, 1]  # Берем сумму выплаты

xgb_params = {'n_estimators': 10000,
              'learning_rate': 0.05,
              'subsample': 0.926,
              'colsample_bytree': 0.84,
              'max_depth': 2,
              'booster': 'gbtree',
              'reg_lambda': 35.1,
              'reg_alpha': 34.9,
              'random_state': 42,
              'n_jobs': -1}
XGBRe = XGBRegressor(**xgb_params)
XGBRe.fit(X_train_1, np.log(y_train_sum+1))
y_pred_sum = np.exp(XGBRe.predict(X_test_1))-1  # исправил

[16:57:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [13]:
y_new = pd.DataFrame({"месяц выплаты": y_pred_month,
                      "сумма выплаты": y_pred_sum,	"тип выплаты": y_pred_type})
y_new.to_excel('/content/drive/MyDrive/TЗ_Будущее/y_new.xlsx')